#### Add wholesale prices to agent file

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# load the county to wholesale price mapping, the dgen county fips mapping, and the agent file
cty_prices = pd.read_pickle("../../../data/stanford_files/data_county_to_wholesale.pkl")
county_map = pd.read_csv("../../../data/dgen_county_fips_mapping.csv", dtype={'geoid10':'str', 'county_id':'str'})
agents = pd.read_pickle("../../input_agents/agent_df_base_res_national_updated_rates.pkl").reset_index(drop = False)

In [ ]:
# Convert the dict of county wholesale prices to a df for the join
prices_df = pd.DataFrame(
    [(str(k), np.asarray(v, dtype=float)) for k, v in cty_prices.items()],
    columns=["geoid10", "wholesale_prices"]
)

# change agent df county_id column to str
agents['county_id'] = agents['county_id'].astype(str)

# Change Oglala Lakota County fips code for join
county_map['geoid10'] = np.where(county_map['geoid10'] == '46113', '46102', county_map['geoid10'])

# Merge to the county_map
county_prices = county_map[['geoid10', 'county_id']].merge(prices_df, on = ['geoid10'], how = "left")

# Merge the prices to the agent df
agents_prices = agents.merge(county_prices[['county_id', 'wholesale_prices']], on = ['county_id'], how = 'left')

In [ ]:
agents_prices.set_index("agent_id").to_pickle("../../input_agents/agent_df_base_res_national_updated_wholesale_prices.pkl")